In [ ]:
# default_exp design

# Design
> Design sequences for regions of interest

In [ ]:
#export
import pandas as pd
from sgrna_designer import ensembl
from sgrna_designer import tile

In [ ]:
# export
def reverse_compliment(seq):
    """Return the reverse compliment of a sequence

    seq: str |

    return: str
    """
    compliment = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    reverse = seq[::-1]
    rev_comp = ''.join([compliment[nt] for nt in reverse])
    return rev_comp

In [ ]:
assert reverse_compliment('ACTG') == 'CAGT'

## Global indexing

Given the global start and end of a sequence (location with the genome) and its orientation, track the
the global position of targetting sgRNAs.

In [ ]:
# export
def calculate_global_position(strand, start, end, relative_position):
    """Calculate the global position of locus in a subsequence

    strand: int [1 or -1], strand of DNA sequence |
    start: int, global start position of sequence |
    end: int, global end position of sequence |
    relative_position: list of int, positions of sgRNAs relative to sequence |

    returns: list of int
    """
    if strand == 1:
        global_position = [start + x for x in relative_position]
    elif strand == -1:
        global_position = [end - x for x in relative_position]
    else:
        raise ValueError("Strand must be 1 or -1")
    return global_position

In [ ]:
region_start = 20100
region_end = 20200
chrom = '1'
sequence = ensembl.get_region_sequence(region_start, region_end, chrom)
sgrna_df = tile.build_sgrna_df(sequence, 30, -6, 3, 4, 20, pams=['CGG', 'AGG', 'TGG', 'GGG'])
sgrna_df['sgrna_global_start'] = calculate_global_position(1, region_start, region_end, sgrna_df['sgrna_relative_start'])
assert(sgrna_df['sgrna_sequence'][0] == ensembl.get_region_sequence(sgrna_df['sgrna_global_start'][0],
                                                                    sgrna_df['sgrna_global_start'][0] + 19,
                                                                    chrom))

Note that the direction is reversed for a sequence on the negative DNA strand

In [ ]:
reverse_sequence = reverse_compliment(sequence)
rev_sgrna_df = tile.build_sgrna_df(reverse_sequence, 30, -6, 3, 4, 20, pams=['CGG', 'AGG', 'TGG', 'GGG'])
rev_sgrna_df['sgrna_global_start'] = calculate_global_position(-1, region_start, region_end,
                                                               rev_sgrna_df['sgrna_relative_start'])
assert(rev_sgrna_df['sgrna_sequence'][0] == reverse_compliment(ensembl.get_region_sequence(
    rev_sgrna_df['sgrna_global_start'][0] - 19,
    rev_sgrna_df['sgrna_global_start'][0],
    chrom)))

In [ ]:
# export
def traverse_global_position(strand, reference_position, distance):
    """Move from one global position to another

    strand: int [1 or -1], strand of DNA sequence |
    reference_position: list of int, global starting position |
    distance: int, distance to move 5' to 3' on either strand |

    returns: list of int
    """
    if strand == 1:
        new_position = [x + distance for x in reference_position]
    elif strand == -1:
        new_position = [x - distance for x in reference_position]
    else:
        raise ValueError("Strand must be 1 or -1")
    return new_position

In [ ]:
sgrna_positions = [4, 8]
for pos in sgrna_positions:
    sgrna_df['sgrna_global_' + str(pos)] = traverse_global_position(1, sgrna_df['sgrna_global_start'], pos-1)
assert (sgrna_df['sgrna_sequence'][0][3:8] == ensembl.get_region_sequence(sgrna_df['sgrna_global_4'][0],
                                                                          sgrna_df['sgrna_global_8'][0],
                                                                          chrom))

Again the direction is reversed for a sequence on the negative DNA strand

In [ ]:
for pos in sgrna_positions:
    rev_sgrna_df['sgrna_global_' + str(pos)] = traverse_global_position(-1, rev_sgrna_df['sgrna_global_start'], pos-1)
assert rev_sgrna_df['sgrna_sequence'][0][3:8] == reverse_compliment(ensembl.get_region_sequence(
    rev_sgrna_df['sgrna_global_8'][0],
    rev_sgrna_df['sgrna_global_4'][0],
    chrom))

In [ ]:
# export
def get_sgrna_global_indices(sgrna_df, seq_start, seq_end, strand, sg_positions=None):
    """Take an sgrna_df and return the globabl positions of elements of the sgRNA

    sgrna_df: Dataframe from `build_sgrna_df` |
    seq_start: int, starting index of sequence |
    seq_end: int, ending index of sequence |
    strand: int [1 or -1], strand of DNA sequence |
    sg_positions: list, positions within the sgRNA to annotate (e.g. [4,8] for nucleotides 4 and 8 of the sgRNA) |

    returns: DataFrame
    """
    indexed_sgrna_df = sgrna_df.copy()
    indexed_sgrna_df['sgrna_global_start'] = calculate_global_position(strand, seq_start, seq_end,
                                                                       indexed_sgrna_df['sgrna_relative_start'])
    if sg_positions is not None:
        for pos in sg_positions:
            indexed_sgrna_df['sgrna_global_' + str(pos)] = traverse_global_position(strand,
                                                                                    indexed_sgrna_df['sgrna_global_start'],
                                                                                    pos-1)
    indexed_sgrna_df = indexed_sgrna_df.drop('sgrna_relative_start', axis=1)
    return indexed_sgrna_df

## Building a full sgRNA design run

In [ ]:
# export
def get_trainscript_region_info(transcript_info, region_parent, region):
    """Return dictionaries for specified regions of interest

    transcript_info: dict, returned by https://rest.ensembl.org/documentation/info/lookup |
    region_parent: str, first level key in transcript_info eg. UTR |
    region: str, second level key in trancript_info eg. three_prime_UTR |

    returns: list of dict
    """
    if region_parent in transcript_info.keys():
        parent_info = transcript_info[region_parent]
        regions = []
        for r in parent_info:
            if r['object_type'] == region:
                regions.append(r)
    else:
        raise ValueError(region_parent + ' or ' + region + 'element could not be identified')
    return regions

def get_target_regions_df(target_transcripts, region_parent, region,
                          expand_3prime, expand_5prime):
    """For a list of transcripts return the locations for a specific region (e.g. three_prime_UTR)

    target_transcripts: list of str
    region_parent: str, first level key in transcript_info eg. UTR |
    region: str, second level key in trancript_info eg. three_prime_UTR |
    expand_3prime: int, length to expand region in 3' direction |
    expand_5prime: int, length to expand region in 5' direction |

    returns: DataFrame of target regions
    """
    target_transcript_info = []
    for transcript in target_transcripts:
        transcript_info = ensembl.get_ensembl_id_information(transcript)
        region_info = get_trainscript_region_info(transcript_info, region_parent, region)
        region_info_df = pd.DataFrame(region_info)
        region_info_df['transcript_id'] = transcript
        target_transcript_info.append(region_info_df)
    target_regions_df = pd.concat(target_transcript_info)
    target_regions_df['expanded_start'] = target_regions_df['start'] - expand_3prime
    target_regions_df['expanded_end'] = target_regions_df['end'] + expand_5prime
    # We keep track of region_pos for merging with the target_sequence_df
    target_regions_df['region_pos'] = target_regions_df.apply(lambda row:
                                                              ensembl.create_region_str(row['expanded_start'],
                                                                                        row['expanded_end'],
                                                                                        row['seq_region_name']),
                                                              axis=1)
    return target_regions_df

In [ ]:
regions = get_target_regions_df(['ENST00000381577'], 'UTR', 'five_prime_UTR', 30, 30)
assert regions.shape[0] == 2
assert regions['start'][0] == 5450542
assert regions['end'][0] == 5450596

In [ ]:
# export
def get_target_regions_sequences(target_regions_df):
    """Get sequences from a DataFrame of target regions

    target_regions_df: DataFrame from `get_target_regions_df`

    returns: DataFrame
    """
    target_sequences = ensembl.post_region_sequences(target_regions_df['expanded_start'],
                                                     target_regions_df['expanded_end'],
                                                     target_regions_df['seq_region_name'])
    target_sequence_df = pd.DataFrame(target_sequences)
    return target_sequence_df

In [ ]:
target_sequences_df = get_target_regions_sequences(regions)
assert 'AGTTCTGCGCAGCTTCCCGAGGCTCCGCACCAGCCGCGCTTCTGTCCGCCTGCAG' in target_sequences_df['seq'][0]

In [ ]:
def merge_target_region_sequence_df(target_regions_df, target_sequences_df):
    """Merge target regions and target sequences dataframes

    target_regions_df:  DataFrame, from `get_target_regions_df` |
    target_sequences_df: DataFrame, from `get_target_regions_sequences` |

    returns: DataFrame
    """
    target_region_seq_df = (target_regions_df
                            .rename({'id': 'region_id'}, axis=1)
                            .merge(target_sequences_df
                                   .rename({'query': 'region_pos',
                                            'id': 'region_pos_id'}, axis=1), how='inner', on='region_pos'))
    return target_region_seq_df

In [ ]:
target_region_seq_df = merge_target_region_sequence_df(regions, target_sequences_df)
target_region_seq_df

,db_type,Parent,region_id,end,object_type,assembly_name,species,start,source,seq_region_name,strand,transcript_id,expanded_start,expanded_end,region_pos,region_pos_id,molecule,seq
0,core,ENST00000381577,ENST00000381577,5450596,five_prime_UTR,GRCh38,homo_sapiens,5450542,ensembl_havana,9,1,ENST00000381577,5450512,5450626,9:5450512..5450626,chromosome:GRCh38:9:5450512:5450626:1,dna,CTGAGCAGCTGGCGCGTCCCGCGCGGCCCCAGTTCTGCGCAGCTTC...
1,core,ENST00000381577,ENST00000381577,5456113,five_prime_UTR,GRCh38,homo_sapiens,5456100,ensembl_havana,9,1,ENST00000381577,5456070,5456143,9:5456070..5456143,chromosome:GRCh38:9:5456070:5456143:1,dna,CAGTCTTCTTTTCGTGTTTTCCATAATTAGGGCATTCCAGAAAGAT...


In [ ]:
def get_transcript_sgrnas(target_region_seq_df, context_len, pam_start, pam_len,
                          sgrna_start, sgrna_len, pams, sg_positions):
    """Get all sgRNAs targeting transcript regions

    target_region_seq_df: DataFrame, from `merge_target_region_sequence_df`
    context_len: int, length of context sequence |
    pam_start: int, position of PAM start relative to the context sequence |
    pam_len: int, length of PAM |
    sgrna_start: int, position of sgRNA relative to context sequence |
    sgrna_len: length of sgRNA sequence |
    pams: list or None, PAMs to design against |
    sg_positions: list of int, positions within the sgRNA to annotate
    (e.g. [4,8] for nucleotides 4 and 8 of the sgRNA) |

    returns DataFrame
    """
    sgrna_df_list = []
    meta_columns = ['object_type', 'strand', 'transcript_id', 'seq_region_name', 'region_id', 'start', 'end']
    for i, row in target_region_seq_df.iterrows():
        seq_start = row['expanded_start']
        seq_end = row['expanded_end']
        sequence = row['seq']
        # Sequences on the positive strand
        pos_sgrna_df = tile.build_sgrna_df(sequence, context_len=context_len, pam_start=pam_start,
                                           pam_len=pam_len, sgrna_start=sgrna_start,
                                           sgrna_len=sgrna_len, pams=pams)
        pos_sgrna_df = get_sgrna_global_indices(pos_sgrna_df, seq_start, seq_end, 1, sg_positions)
        # assuming the target_region_seq_df is oriented on the positive sgRNA strand
        pos_sgrna_df['sgrna_strand'] = 1
        # Sequences on the negative strand
        rev_comp_seq = reverse_compliment(sequence)
        neg_sgrna_df = tile.build_sgrna_df(rev_comp_seq, context_len=context_len, pam_start=pam_start,
                                           pam_len=pam_len, sgrna_start=sgrna_start,
                                           sgrna_len=sgrna_len, pams=pams)
        neg_sgrna_df = get_sgrna_global_indices(neg_sgrna_df, seq_start, seq_end, -1, sg_positions)
        neg_sgrna_df['sgrna_strand'] = -1
        # Combine and filter sgrna_dfs
        sgrna_df = pd.concat([pos_sgrna_df, neg_sgrna_df])
        for col in meta_columns:
            sgrna_df[col] = row[col]
        sgrna_df_list.append(sgrna_df)
    concatenated_sgrna_dfs = (pd.concat(sgrna_df_list)
                              .rename({'strand': 'transcript_strand',
                                       'start': 'region_start',
                                       'end': 'region_end',
                                       'seq_region_name': 'chromosome'}, axis=1))
    return concatenated_sgrna_dfs

In [ ]:
transcript_sgrna_df = get_transcript_sgrnas(target_region_seq_df, context_len=30, pam_start=-6,
                                            pam_len=3, sgrna_start=4, sgrna_len=20, pams=['AGG', 'CGG', 'TGG', 'GGG'],
                                            sg_positions=[17, 18])
assert 'CCGCGCTTCTGTCCGCCTGC' in transcript_sgrna_df['sgrna_sequence'].to_list()

In [ ]:
# export
def filter_sgrnas_by_region(transcript_sgrna_df, sg_positions):
    """Filter for sgRNAs such that the specified sgRNA positions are within the region of interest

    transcript_sgrna_df: DataFrame, from`get_transcript_sgrnas`
    sg_positions: list of int, positions within the sgRNA to annotate
    (e.g. [4,8] for nucleotides 4 and 8 of the sgRNA) |

    returns: DataFrame, filtered for sgRNAs that fall within the target region
    """
    global_pos_cols = []
    for pos in sg_positions:
        global_pos_cols.append('sgrna_global_' + str(pos))
    filtered_sgrna_df = transcript_sgrna_df[
        ((transcript_sgrna_df[global_pos_cols].min(axis=1) >= transcript_sgrna_df['region_start']) &
         (transcript_sgrna_df[global_pos_cols].min(axis=1) <= transcript_sgrna_df['region_end'])) |
        ((transcript_sgrna_df[global_pos_cols].max(axis=1) >= transcript_sgrna_df['region_start']) &
         (transcript_sgrna_df[global_pos_cols].max(axis=1) <= transcript_sgrna_df['region_end']))
    ].copy()
    return filtered_sgrna_df

In [ ]:
filtered_sgrnas = filter_sgrnas_by_region(transcript_sgrna_df, [17, 18])
assert filtered_sgrnas.shape[0] < transcript_sgrna_df.shape[0]

In [ ]:
# export
def design_sgrna_tiling_library(target_transcripts, region_parent, region,
                                expand_3prime, expand_5prime, context_len,
                                pam_start, pam_len, sgrna_start, sgrna_len,
                                pams, sg_positions):
    """Design sgRNAs tiling transcript regions

    target_transcripts: list of str
    region_parent: str, first level key in transcript_info eg. UTR |
    region: str, second level key in trancript_info eg. three_prime_UTR |
    expand_3prime: int, length to expand region in 3' direction |
    expand_5prime: int, length to expand region in 5' direction |
    context_len: int, length of context sequence |
    pam_start: int, position of PAM start relative to the context sequence |
    pam_len: int, length of PAM |
    sgrna_start: int, position of sgRNA relative to context sequence |
    sgrna_len: length of sgRNA sequence |
    pams: list or None, PAMs to design against |
    sg_positions: list of int, positions within the sgRNA to annotate
    (e.g. [4,8] for nucleotides 4 and 8 of the sgRNA) |

    returns: DataFrame, sgRNAs for transcript regions of interest
    """
    target_regions_df = get_target_regions_df(target_transcripts=target_transcripts, region_parent=region_parent,
                                              region=region, expand_3prime=expand_3prime,
                                              expand_5prime=expand_5prime)
    target_sequences_df = get_target_regions_sequences(target_regions_df)
    target_region_seq_df = merge_target_region_sequence_df(target_regions_df, target_sequences_df)
    transcript_sgrna_df = get_transcript_sgrnas(target_region_seq_df, context_len=context_len, pam_start=pam_start,
                                                pam_len=pam_len, sgrna_start=sgrna_start, sgrna_len=sgrna_len,
                                                pams=pams, sg_positions=sg_positions)
    filtered_sgrnas = filter_sgrnas_by_region(transcript_sgrna_df, sg_positions)
    return filtered_sgrnas

In [ ]:
target_transcripts = ['ENST00000381577', 'ENST00000644969']
exon_tiling_library = design_sgrna_tiling_library(target_transcripts, region_parent='Exon', region='Exon',
                                                  expand_3prime=30, expand_5prime=30, context_len=30,
                                                  pam_start=-6, pam_len=3, sgrna_start=4, sgrna_len=20,
                                                  pams=['AGG', 'CGG', 'TGG', 'GGG'], sg_positions=[17, 18])
utr3_tiling_library = design_sgrna_tiling_library(target_transcripts, region_parent='UTR', region='three_prime_UTR',
                                                  expand_3prime=30, expand_5prime=30, context_len=30,
                                                  pam_start=-6, pam_len=3, sgrna_start=4, sgrna_len=20,
                                                  pams=['AGG', 'CGG', 'TGG', 'GGG'], sg_positions=[17, 18])
utr5_tiling_library = design_sgrna_tiling_library(target_transcripts, region_parent='UTR', region='five_prime_UTR',
                                                  expand_3prime=30, expand_5prime=30, context_len=30,
                                                  pam_start=-6, pam_len=3, sgrna_start=4, sgrna_len=20,
                                                  pams=['AGG', 'CGG', 'TGG', 'GGG'], sg_positions=[17, 18])
# This is an approximation, as some guides are actually at the UTR/CDS border
cds_tiling_library = exon_tiling_library[~exon_tiling_library['sgrna_sequence']
                                         .isin(utr3_tiling_library['sgrna_sequence'].to_list() +
                                               utr5_tiling_library['sgrna_sequence'].to_list())].copy()
crispick_designs = pd.read_table('data/example_cas9_sgrna-designs.txt')
assert (set(crispick_designs['sgRNA Sequence']) - {'TAGGGCATTCCAGAAAGATG'} ==
        set(cds_tiling_library['sgrna_sequence']))
cds_tiling_library

,context_sequence,pam_sequence,sgrna_sequence,sgrna_global_start,sgrna_global_17,sgrna_global_18,sgrna_strand,object_type,transcript_strand,transcript_id,chromosome,region_id,region_start,region_end
3,GCTGTCTTTATATTCATGACCTACTGGCAT,TGG,TCTTTATATTCATGACCTAC,5456130.0,5456146.0,5456147.0,1,Exon,1,ENST00000381577,9,ENSE00000813155,5456100,5456165
4,CATGACCTACTGGCATTTGCTGAACGGTAA,CGG,ACCTACTGGCATTTGCTGAA,5456144.0,5456160.0,5456161.0,1,Exon,1,ENST00000381577,9,ENSE00000813155,5456100,5456165
0,TCTTACCGTTCAGCAAATGCCAGTAGGTCA,AGG,ACCGTTCAGCAAATGCCAGT,5456167.0,5456151.0,5456150.0,-1,Exon,1,ENST00000381577,9,ENSE00000813155,5456100,5456165
0,CTTTCTTTTTAGCATTTACTGTCACGGTTC,CGG,CTTTTTAGCATTTACTGTCA,5457071.0,5457087.0,5457088.0,1,Exon,1,ENST00000381577,9,ENSE00003541119,5457079,5457420
1,TAGCATTTACTGTCACGGTTCCCAAGGACC,AGG,ATTTACTGTCACGGTTCCCA,5457080.0,5457096.0,5457097.0,1,Exon,1,ENST00000381577,9,ENSE00003541119,5457079,5457420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,TCTCCAAAAACACCCATCCAGGCAGGGGGA,GGG,CAAAAACACCCATCCAGGCA,140734646.0,140734630.0,140734629.0,-1,Exon,-1,ENST00000644969,7,ENSE00001907699,140734617,140734770
8,CTCCAAAAACACCCATCCAGGCAGGGGGAT,GGG,AAAAACACCCATCCAGGCAG,140734645.0,140734629.0,140734628.0,-1,Exon,-1,ENST00000644969,7,ENSE00001907699,140734617,140734770
9,TCCAAAAACACCCATCCAGGCAGGGGGATA,GGG,AAAACACCCATCCAGGCAGG,140734644.0,140734628.0,140734627.0,-1,Exon,-1,ENST00000644969,7,ENSE00001907699,140734617,140734770
10,AACACCCATCCAGGCAGGGGGATATGGTGC,TGG,CCCATCCAGGCAGGGGGATA,140734638.0,140734622.0,140734621.0,-1,Exon,-1,ENST00000644969,7,ENSE00001907699,140734617,140734770
